In [1]:
import sys

sys.path.append("../..")
sys.path.append("..")

import numpy as np
import pandas as pd
import random
import torch
from torch_rechub.models.multi_task import SharedBottom, ESMM, MMOE, PLE, AITM
from torch_rechub.trainers import MTLTrainer
from torch_rechub.basic.features import DenseFeature, SparseFeature
from torch_rechub.utils.data import DataGenerator

In [2]:
config_path = '/tmp/gaizhy/applied_algorithm/rampUp/applied_ag_utils'
# warnings.filterwarnings("ignore")

sys.path.append(config_path)  # append上三级目录，使之能加载dzj_toolkits包
from gaizhy_utils import logger_initialize
logger = logger_initialize('MMOE_rechub_train.log')

random.seed(2023)
np.random.seed(2023)
pd.np.random.seed(2023)
torch.manual_seed(2023)
torch.cuda.manual_seed_all(2023)
torch.backends.cudnn.deterministic = True

/tmp/ipykernel_2312562/2090339189.py:10: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  pd.np.random.seed(2023)


In [ ]:
# import random

# def datamaker(root_path,train_data_name,test_data_name,val_data_name,change_data_rate=0.05):
#     origin_data_path = {'train':root_path+train_data_name,
#                         'test':root_path+test_data_name,
#                         'val':root_path+val_data_name
#                        }
#     for i in origin_data_path:
#         data_path = origin_data_path[i]
#         data = pd.read_csv(data_path) # 读取数据
#         data.insert(1,'carts',0) # 插入标签数据
        
#         # 强制让购买行为下的加购行为标签为1
#         data.loc[(data['click']==1)&(data['purchase']==1),'carts']=1

#         # 找出可对加购标签进行修改的index
#         change_index_list = list(data.loc[(data['click']==1)&(data['purchase']!=1),'carts'].index)

#         # 按比例随机选取可进行修改的加购标签index
#         change_number = round(len(change_index_list)*change_data_rate)
#         selsct_change_index = random.sample(change_index_list,change_number)

#         # 对随机选中的数据修改加购行为标签
#         data.loc[selsct_change_index,'carts']=1
        
#         # 保存数据到原目录
#         data.to_csv(data_path.split('.csv')[0]+'_add_data.csv',index = False)
        
#         print('%s数据集，可修改的点击标签有%d个，修改比例为%.2f，修改的加购行为标签数量为%d个'%(i,len(change_index_list),change_data_rate,change_number))
        
# root_path = '/tmp/gaizhy/aliccp/'
# train_data_name = 'ali_ccp_train.csv'
# test_data_name = 'ali_ccp_test.csv'
# val_data_name = 'ali_ccp_val.csv'
# change_data_rate = 0.05
# datamaker(root_path,train_data_name,test_data_name,val_data_name)

In [3]:
data_path = '/tmp/gaizhy/aliccp'

# df_train_origin = pd.read_csv(data_path + '/ali_ccp_train.csv', nrows=None)
# df_val = pd.read_csv(data_path + '/ali_ccp_val.csv', nrows=None)
# df_test = pd.read_csv(data_path + '/ali_ccp_test.csv', nrows=None)

# df_train_origin = pd.read_csv(data_path + '/ali_ccp_train_with_carts.csv', nrows=None)
# df_val = pd.read_csv(data_path + '/ali_ccp_val_with_carts.csv', nrows=None)
# df_test = pd.read_csv(data_path + '/ali_ccp_test_with_carts.csv', nrows=None)

df_train_origin = pd.read_csv(data_path + '/ali_ccp_train_with_carts.csv', nrows=None)
df_val = pd.read_csv(data_path + '/ali_ccp_val_with_carts.csv', nrows=None)
df_test = pd.read_csv(data_path + '/ali_ccp_test_with_carts.csv', nrows=None)

In [4]:
df_train_origin.drop(columns=['Unnamed: 0', 'index', 'index.1', 'p_purchase', 'p_click'],inplace=True)
df_val.drop(columns=['Unnamed: 0', 'index', 'index.1', 'p_purchase', 'p_click'], inplace=True)
df_test.drop(columns=['Unnamed: 0', 'index', 'index.1', 'p_purchase', 'p_click'], inplace=True)

In [ ]:
df_train_origin.loc[:,['click', 'purchase']].value_counts(dropna=False)

In [ ]:
train_data_bal.loc[:,['click', 'purchase']].value_counts(dropna=False)

In [5]:
df_train_origin.loc[:,['click','carts','purchase']].value_counts(dropna=False)

click  carts  purchase
0      0      0           40655649
1      0      0            1517701
       1      0             117753
              1               8802
dtype: int64

In [ ]:
train_data_pos = df_train_origin[df_train_origin['click']==1]
print(train_data_pos.shape)
train_data_neg = df_train_origin[df_train_origin['click']==0]
train_data_neg = train_data_neg.sample(n=16442560)
print(train_data_neg.shape)
train_data_bal = pd.concat([train_data_pos,train_data_neg], axis=0)
print(train_data_bal.shape)

In [6]:
df_train = df_train_origin
# df_train = train_data_bal

In [7]:
df_train.head(1)

,click,purchase,101,121,122,124,125,126,127,128,...,150_14,D109_14,D110_14,D127_14,D150_14,D508,D509,D702,D853,carts
0,0,0,1,1,1,1,1,0,1,1,...,1,0.4734,0.562,0.0856,0.1902,0.07556,0.0,0.0,0.0,0


In [8]:
print("train : val : test = %d %d %d" % (len(df_train), len(df_val), len(df_test)))
train_idx, val_idx = df_train.shape[0], df_train.shape[0] + df_val.shape[0]
data = pd.concat([df_train, df_val, df_test], axis=0)
#task 1 (as cvr): main task, purchase prediction
#task 2(as ctr): auxiliary task, click prediction

# data.rename(columns={'purchase': 'cvr_label', 'click': 'ctr_label'}, inplace=True)
data.rename(columns={'purchase': 'cvr_label', 'carts': 'car_label','click': 'ctr_label'}, inplace=True)
data["ctcvr_label"] = data['cvr_label'] * data['ctr_label']

col_names = data.columns.values.tolist()
dense_cols = ['D109_14', 'D110_14', 'D127_14', 'D150_14', 'D508', 'D509', 'D702', 'D853']
# sparse_cols = [col for col in col_names if col not in dense_cols and col not in ['cvr_label', 'ctr_label', 'ctcvr_label']]
sparse_cols = [col for col in col_names if col not in dense_cols and col not in ['cvr_label', 'ctr_label', 'ctcvr_label', 'car_label']]
print("sparse cols:%d dense cols:%d" % (len(sparse_cols), len(dense_cols)))

# label_cols = ['cvr_label', 'ctr_label']  #the order of labels can be any
label_cols = ['cvr_label', 'car_label', 'ctr_label']  #the order of labels can be any
used_cols = sparse_cols + dense_cols
features = [SparseFeature(col, data[col].max()+1, embed_dim=4)for col in sparse_cols] \
           + [DenseFeature(col) for col in dense_cols]


x_train, y_train = {name: data[name].values[:train_idx] for name in used_cols}, data[label_cols].values[:train_idx]
x_val, y_val = {name: data[name].values[train_idx:val_idx] for name in used_cols}, data[label_cols].values[train_idx:val_idx]
x_test, y_test = {name: data[name].values[val_idx:] for name in used_cols}, data[label_cols].values[val_idx:]

train : val : test = 42299905 21508307 21508307
sparse cols:23 dense cols:8


In [17]:
epoch, learning_rate, batch_size, weight_decay, device, save_dir = 5, 1e-3, 8192, 1e-4, 'cuda:0', './'

# elif model_name == "MMOE":
# features, x_train, y_train, x_val, y_val, x_test, y_test = get_ali_ccp_data_dict(model_name)
# task_types = ["classification", "classification"]
task_types = ["classification", "classification", "classification"]

dg = DataGenerator(x_train, y_train)
train_dataloader, val_dataloader, test_dataloader = dg.generate_dataloader(x_val=x_val, y_val=y_val, x_test=x_test, y_test=y_test, batch_size=batch_size)

In [20]:
# model = MMOE(features, task_types, 8, expert_params={"dims": [16]}, tower_params_list=[{"dims": [8]}, {"dims": [8]}])
# model = MMOE(features, task_types, 8, expert_params={"dims": [16]}, tower_params_list=[{"dims": [8]}, {"dims": [8]}, {"dims": [8]}])
# model = PLE(features, task_types, n_level=1, n_expert_specific=2, n_expert_shared=1, expert_params={"dims": [16], "dropout":0.05}, tower_params_list=[{"dims": [8]}, {"dims": [8]}])
model = PLE(features, task_types, n_level=1, n_expert_specific=2, n_expert_shared=1, expert_params={"dims": [16], "dropout":0.05}, tower_params_list=[{"dims": [8]}, {"dims": [8]}, {"dims": [8]}])

#adaptive weight loss:
#mtl_trainer = MTLTrainer(model, task_types=task_types, optimizer_params={"lr": learning_rate, "weight_decay": weight_decay}, adaptive_params={"method": "uwl"}, n_epoch=epoch, earlystop_patience=10, device=device, model_path=save_dir)

mtl_trainer = MTLTrainer(model, task_types=task_types, 
    optimizer_params={"lr": learning_rate, "weight_decay": weight_decay}, 
    n_epoch=epoch, earlystop_patience=10, device=device, gpus=['cuda:0', 'cuda:1'], model_path=save_dir)

parallel running on these gpus: ['cuda:0', 'cuda:1']


In [ ]:
mtl_trainer.fit(train_dataloader, val_dataloader)

train:  17%|█▋        | 883/5164 [01:14<06:01, 11.83it/s] 

In [ ]:
# torch.save(mtl_trainer.model.state_dict(), 'MMOE_click&purchase.pt')
# torch.save(mtl_trainer.model.state_dict(), 'MMOE_click&carts&purchase.pt')
torch.save(mtl_trainer.model.state_dict(), 'PLE_click&carts&purchase.pt')

In [ ]:
mtl_trainer.model.load_state_dict(torch.load('MMOE_click&purchase.pt'))
mtl_trainer.model.to(mtl_trainer.device)

In [ ]:
auc = mtl_trainer.evaluate(mtl_trainer.model, test_dataloader)
logger.info(f'lr: {learning_rate}, bs: {batch_size}, wd: {weight_decay}')
logger.info(f'test auc: {auc}')

In [ ]:
learning_rates = [1e-3, 5e-4, 2.5e-4, 2e-3, 4e-3]
batch_sizes = [1024, 512, 256, 2048, 4096]
weight_decaies = [1e-4, 5e-5, 2.5e-5, 2e-4, 4e-4]

In [ ]:
# parser.add_argument('--epoch', type=int, default=1)
# parser.add_argument('--learning_rate', type=float, default=1e-3)
# parser.add_argument('--batch_size', type=int, default=1024)
# parser.add_argument('--weight_decay', type=float, default=1e-4)
# parser.add_argument('--device', default='cpu')  #cuda:0
# parser.add_argument('--save_dir', default='./')
# parser.add_argument('--seed', type=int, default=2022)
for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for weight_decay in weight_decaies:
            # epoch, learning_rate, batch_size, weight_decay, device, save_dir, seed = 10, 1e-3, 1024, 1e-4, 'cuda:0', './', 2022
            epoch, device, save_dir, seed = 10, 'cuda:0', './', 2022

            torch.manual_seed(seed)
            # elif model_name == "MMOE":
            # features, x_train, y_train, x_val, y_val, x_test, y_test = get_ali_ccp_data_dict(model_name)
            task_types = ["classification", "classification"]
            model = MMOE(features, task_types, 8, expert_params={"dims": [16]}, tower_params_list=[{"dims": [8]}, {"dims": [8]}])

            dg = DataGenerator(x_train, y_train)
            train_dataloader, val_dataloader, test_dataloader = dg.generate_dataloader(x_val=x_val, y_val=y_val, x_test=x_test, y_test=y_test, batch_size=batch_size)

            #adaptive weight loss:
            mtl_trainer = MTLTrainer(model, task_types=task_types, 
                optimizer_params={"lr": learning_rate, "weight_decay": weight_decay}, 
                n_epoch=epoch, earlystop_patience=10, device=device, gpus=['cuda:0', 'cuda:1'], model_path=save_dir)
            mtl_trainer.fit(train_dataloader, val_dataloader)
            auc = mtl_trainer.evaluate(mtl_trainer.model, test_dataloader)
            print(f'lr: {learning_rate}, bs: {batch_size}, wd: {weight_decay}')
            print(f'test auc: {auc}')

In [ ]:
def main(model_name, epoch, learning_rate, batch_size, weight_decay, device, save_dir, seed):
    torch.manual_seed(seed)
    if model_name == "SharedBottom":
        features, x_train, y_train, x_val, y_val, x_test, y_test = get_ali_ccp_data_dict(model_name)
        task_types = ["classification", "classification"]
        model = SharedBottom(features, task_types, bottom_params={"dims": [117]}, tower_params_list=[{"dims": [8]}, {"dims": [8]}])
    elif model_name == "ESMM":
        user_features, item_features, x_train, y_train, x_val, y_val, x_test, y_test = get_ali_ccp_data_dict(model_name)
        task_types = ["classification", "classification", "classification"]  #cvr,ctr,ctcvr
        model = ESMM(user_features, item_features, cvr_params={"dims": [16, 8]}, ctr_params={"dims": [16, 8]})
    elif model_name == "MMOE":
        features, x_train, y_train, x_val, y_val, x_test, y_test = get_ali_ccp_data_dict(model_name)
        task_types = ["classification", "classification"]
        model = MMOE(features, task_types, 8, expert_params={"dims": [16]}, tower_params_list=[{"dims": [8]}, {"dims": [8]}])
    elif model_name == "PLE":
        features, x_train, y_train, x_val, y_val, x_test, y_test = get_ali_ccp_data_dict(model_name)
        task_types = ["classification", "classification"]
        model = PLE(features, task_types, n_level=1, n_expert_specific=2, n_expert_shared=1, expert_params={"dims": [16]}, tower_params_list=[{"dims": [8]}, {"dims": [8]}])
    elif model_name == "AITM":
        task_types = ["classification", "classification"]
        features, x_train, y_train, x_val, y_val, x_test, y_test = get_ali_ccp_data_dict(model_name)
        model = AITM(features, 2, bottom_params={"dims": [32, 16]}, tower_params_list=[{"dims": [8]}, {"dims": [8]}])

    dg = DataGenerator(x_train, y_train)
    train_dataloader, val_dataloader, test_dataloader = dg.generate_dataloader(x_val=x_val, y_val=y_val, x_test=x_test, y_test=y_test, batch_size=batch_size)

    #adaptive weight loss:
    #mtl_trainer = MTLTrainer(model, task_types=task_types, optimizer_params={"lr": learning_rate, "weight_decay": weight_decay}, adaptive_params={"method": "uwl"}, n_epoch=epoch, earlystop_patience=10, device=device, model_path=save_dir)

    mtl_trainer = MTLTrainer(model, task_types=task_types, optimizer_params={"lr": learning_rate, "weight_decay": weight_decay}, n_epoch=epoch, earlystop_patience=30, device=device, model_path=save_dir)
    mtl_trainer.fit(train_dataloader, val_dataloader)
    auc = mtl_trainer.evaluate(mtl_trainer.model, test_dataloader)
    print(f'test auc: {auc}')


if __name__ == '__main__':
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_name', default='SharedBottom')
    parser.add_argument('--epoch', type=int, default=1)
    parser.add_argument('--learning_rate', type=float, default=1e-3)
    parser.add_argument('--batch_size', type=int, default=1024)
    parser.add_argument('--weight_decay', type=float, default=1e-4)
    parser.add_argument('--device', default='cpu')  #cuda:0
    parser.add_argument('--save_dir', default='./')
    parser.add_argument('--seed', type=int, default=2022)

    args = parser.parse_args()
    main(args.model_name, args.epoch, args.learning_rate, args.batch_size, args.weight_decay, args.device, args.save_dir, args.seed)
"""
python run_ali_ccp_multi_task.py --model_name SharedBottom
python run_ali_ccp_multi_task.py --model_name ESMM
python run_ali_ccp_multi_task.py --model_name MMOE
python run_ali_ccp_multi_task.py --model_name PLE
python run_ali_ccp_multi_task.py --model_name AITM
"""

### 3、carts数据集生成

In [ ]:
df_train_origin['carts'] = 0
df_val['carts'] = 0
df_test['carts'] = 0

df_train_origin['index'] = df_train_origin.index
df_val['index'] = df_val.index
df_test['index'] = df_test.index

In [ ]:
import tqdm
model, data_loader = mtl_trainer.model, val_dataloader

model.eval()
predicts = list()
indexies = list()
with torch.no_grad():
    tk0 = tqdm.tqdm(data_loader, desc="predict", smoothing=0, mininterval=1.0)
    for i, (x_dict, ys) in enumerate(tk0):
        x_dict = {k: v.to(mtl_trainer.device) for k, v in x_dict.items()}
        y_preds = model(x_dict)
        predicts.extend(y_preds.tolist())
        indexies.extend(x_dict['index'].tolist())

In [ ]:
df_val_pred = pd.DataFrame(predicts, columns=['p_purchase','p_click'])
print(1)
df_val_index = pd.DataFrame(indexies, columns=['index'])
print(2)
df_val_ipred = pd.concat([df_val_index, df_val_pred], axis=1)
print(3)
df_val_ipred.sort_values('index', inplace=True)
df_val_ipred.reset_index(drop=True, inplace=True)
print(4)
df_val_end = pd.concat([df_val_ipred, df_val], axis=1)
df_val_end.drop(columns=['index'], inplace=True)

In [ ]:
# df_val_end.loc[:,'p_purchase'].quantile(q=0.05)
threshold = df_val_end[(df_val_end['click']==1) & (df_val_end['purchase']==0)]['p_purchase'].quantile(q=0.928) # 13/14=0.928, click,carts,purchase 1,0,0:1,1,0:1,1,1 => 167:13:1

In [ ]:
df_val_end['carts']=0
df_val_end.loc[df_val_end['p_purchase']>=threshold, 'carts']=1
# df_val_end.loc[df_val_end['p_click']>=0.19431710243225098,'carts']=1
df_val_end.loc[df_val_end['purchase']==1,'carts']=1
df_val_end.loc[df_val_end['click']==0,'carts']=0

In [ ]:
df_val_end.loc[:,['click','carts','purchase']].value_counts(dropna=False)

In [ ]:
df_val_end.to_csv('ali_ccp_val_with_carts.csv')

In [ ]:
print